In [ ]:
# !pip install selenium

In [1]:
!docker-compose up -d

time="2024-09-10T08:37:23+01:00" level=warning msg="C:\\Users\\Windows\\Downloads\\Projects\\Hajy8_Grimas_Scraping_Project\\docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 Network hajy8_grimas_scraping_project_default  Creating
 Network hajy8_grimas_scraping_project_default  Created
 Container selenium-hub-hajy8_grimas-project1  Creating
 Container selenium-hub-hajy8_grimas-project1  Created
 Container jupyter-hajy8_grimas-project1  Creating
 Container hajy8_grimas_scraping_project-chrome_3_hajy8_grimas_project1-1  Creating
 Container hajy8_grimas_scraping_project-chrome_1_hajy8_grimas_project1-1  Creating
 Container hajy8_grimas_scraping_project-chrome_4_hajy8_grimas_project1-1  Creating
 Container hajy8_grimas_scraping_project-chrome_6_hajy8_grimas_project1-1  Creating
 Container hajy8_grimas_scraping_project-chrome_5_hajy8_grimas_project1-1  Creating
 Container hajy8_grimas_scraping_project-chrome_2_hajy8_

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time 
import os
import zipfile

class SociteScraping:
    
    def __init__(self, siret = None, proxy=None):
        self.url = 'https://www.societe.com/'
        self.SIRET = siret

        self.with_proxy = True
        if self.with_proxy :
            self.PROXY_HOST = proxy["PROXY_HOST"] # rotating proxy or host
            self.PROXY_PORT = proxy["PROXY_PORT"] # port
            self.PROXY_USER = proxy["PROXY_USER"] # username
            self.PROXY_PASS = proxy["PROXY_PASS"] # password
            self.options = self.get_options_for_proxy()
        else:
            self.options = webdriver.ChromeOptions()
            
        self.with_selenium_grid = True
        if self.with_selenium_grid:
            # IP address and port and server of the Selenium hub and browser options
            self.HUB_HOST = "localhost"
            self.HUB_PORT = 4444
            self.server = f"http://{self.HUB_HOST}:{self.HUB_PORT}/wd/hub"
            self.driver = webdriver.Remote(command_executor=self.server, options=self.options)
        else:
            self.driver = webdriver.Chrome(options=self.options)
            
        # self.start_scraping()
        

    def get_options_for_proxy(self):
        
        manifest_json = """
        {
            "version": "1.0.0",
            "manifest_version": 2,
            "name": "Chrome Proxy",
            "permissions": [
                "proxy",
                "tabs",
                "unlimitedStorage",
                "storage",
                "<all_urls>",
                "webRequest",
                "webRequestBlocking"
            ],
            "background": {
                "scripts": ["background.js"]
            },
            "minimum_chrome_version":"22.0.0"
        }
        """
        
        background_js = """
        var config = {
                mode: "fixed_servers",
                rules: {
                singleProxy: {
                    scheme: "http",
                    host: "%s",
                    port: parseInt(%s)
                },
                bypassList: ["localhost"]
                }
            };
        chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});
        function callbackFn(details) {
            return {
                authCredentials: {
                    username: "%s",
                    password: "%s"
                }
            };
        }
        chrome.webRequest.onAuthRequired.addListener(
                    callbackFn,
                    {urls: ["<all_urls>"]},
                    ['blocking']
        );
        """ % (self.PROXY_HOST, self.PROXY_PORT, self.PROXY_USER, self.PROXY_PASS)
        
        def get_chrome_options(use_proxy=True, user_agent=None):
            chrome_options = webdriver.ChromeOptions()
            if use_proxy:
                pluginfile = 'proxy_auth_plugin.zip'
        
                with zipfile.ZipFile(pluginfile, 'w') as zp:
                    zp.writestr("manifest.json", manifest_json)
                    zp.writestr("background.js", background_js)
                chrome_options.add_extension(pluginfile)
            if user_agent:
                chrome_options.add_argument('--user-agent=%s' % user_agent)
            
            return chrome_options
        return get_chrome_options()
    
    def start_scraping(self):
        try:
            self.driver.get(self.url)
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
             )

            input_search = self.get_element("/html/body/div[2]/header/div[2]/div[2]/div/form/div[1]/div[1]/input")
            if input_search["status"]:
                input_search = input_search["data"]
                input_search.send_keys(self.SIRET)
                input_search.submit()
            else:
                return {"status": False, "data":input_search["data"] }
                
            button_modal = self.get_element('/html/body/div[1]/div/div/div/div/div/div[3]/button[2]')
            if button_modal["status"]:
                button_modal = button_modal["data"]
                button_modal.click()
            else:
                return {"status": False, "data":button_modal["data"] }
            
            
            first_societe = self.get_element('//*[@id="result_deno_societe"]/div/div/a')
            if first_societe["status"]:
                first_societe = first_societe["data"]
                first_societe.click()
            else:
                return {"status": False, "data":first_societe["data"] }


            effectif_elem = self.get_element('//*[@id="trancheeff-histo-description"]')
            capital_elem = self.get_element('//*[@id="capital-histo-description"]')
            gerant_elem = self.get_element('//*[@class="Table leader"]/tbody/tr/td[2]/a/span')

            if not effectif_elem["status"] and not capital_elem["status"] and not gerant_elem["status"] :
                return {"status": False, "data": effectif_elem["status"] }
                
            print(self.SIRET)
            print(f"      {effectif_elem["data"].text if effectif_elem["status"] else None}")
            print(f"      {capital_elem["data"].text if capital_elem["status"] else None}")
            print(f"      {gerant_elem["data"].text if gerant_elem["status"] else None}")

            result = {
                "effectif" : effectif_elem["data"].text if effectif_elem["status"] else None,
                "capital" : capital_elem["data"].text if capital_elem["status"] else None,
                "gerant" : gerant_elem["data"].text if gerant_elem["status"] else None
            }
            
            return {"status": True, "data": result }
            
        except Exception as e:
            print(f"Error : {e}")
            return {"status": False, "data": str(e) }
        finally :
            self.driver.quit()
            
    def get_element(self, path_to_elem, class_=None):
        i = 0
        while i<5:
            try:
                if not class_:
                    elem = self.driver.find_element(By.XPATH, path_to_elem)
                    return {"status": True, "data":elem }
            except Exception as e:
                i += 1
                if i == 5:
                    return {"status": False, "data":str(e) }

In [3]:
import threading
import pandas as pd
import time
from ProxyLoader import ProxyLoader

# Classe ThreadWorker qui permet à chaque thread de traiter une ligne de SIRET
class ThreadWorker(threading.Thread):
    def __init__(self, thread_id, lines, result_lines, error_lines, lock, proxy, lock_grid):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        self.lines = lines
        self.result_lines = result_lines
        self.error_lines = error_lines
        self.lock = lock
        self.proxy = proxy
        self.lock_grid = lock_grid
        self.sociteScraping = None
        print(f"start thread_id : {thread_id} avec proxy : {proxy["PROXY_HOST"]}")
        
    def run(self):
        while True:
            with self.lock:
                if not self.lines:
                    break
                line = self.lines.pop(0)

            try:
                siret = line['SIRET']
                with self.lock_grid :
                    self.sociteScraping = SociteScraping(siret=siret, proxy=self.proxy)
                    time.sleep(1)
                result = self.sociteScraping.start_scraping()
                
                if result["status"]:
                    info = result["data"]
                    line['effectif'] = info['effectif']
                    line['capital'] = info['capital']
                    line['gerant'] = info['gerant']
                    with self.lock:
                        self.result_lines.append(line)
                else:
                    line['error'] = str(result["data"])
                    with self.lock:
                        self.error_lines.append(line)
            except Exception as e:
                line['error'] = str(e)
                with self.lock:
                    self.error_lines.append(line)
                print(e)

def main():
    # Chargement du fichier CSV dans un DataFrame Pandas
    df = pd.read_excel("./results_data/grimas_new_version_1.xlsx")

    # Ajouter la colonne pour les indices d'origine
    df['index_orig'] = df.index

    # Ajouter les colonnes pour les nouvelles valeurs
    df['effectif'] = ''
    df['capital'] = ''
    df['gerant'] = ''

    # Conversion du DataFrame en liste de dictionnaires (chaque ligne est un dict)
    lines = df.to_dict('records')[200:300]

    # Listes pour les lignes traitées correctement et celles avec des erreurs
    result_lines = []
    error_lines = []

    # Verrou pour la synchronisation
    lock = threading.Lock()
    lock_grid = threading.Lock()
    
    file_path = r"./proxies/Webshare 100 proxies.txt"
    proxy_loader = ProxyLoader(file_path)
    
    # Lancement de 10 threads
    threads = []
    for i in range(30):
        worker = ThreadWorker(i, lines, result_lines, error_lines, lock, proxy_loader.get_proxy(i), lock_grid)
        worker.start()
        threads.append(worker)
        
        
    # Attendre la fin de tous les threads
    for thread in threads:
        thread.join()

    # Conversion des listes de résultats et d'erreurs en DataFrames
    df_result = pd.DataFrame(result_lines)
    df_error = pd.DataFrame(error_lines)

    # Trier les DataFrames par l'index d'origine
    df_result = df_result.sort_values(by='index_orig')
    df_error = df_error.sort_values(by='index_orig')

    # Sauvegarde des DataFrames modifiés dans des fichiers CSV
    df_result.to_csv('fichier_avec_info_2.csv', index=False)
    df_error.to_csv('fichier_erreurs_2.csv', index=False)

if __name__ == "__main__":
    main()

start thread_id : 0 avec proxy : 161.123.152.67
start thread_id : 1 avec proxy : 154.73.250.157
start thread_id : 2 avec proxy : 154.73.250.48
start thread_id : 3 avec proxy : 154.73.250.7
start thread_id : 4 avec proxy : 161.123.152.90
start thread_id : 5 avec proxy : 154.73.250.219
start thread_id : 6 avec proxy : 154.73.250.180
start thread_id : 7 avec proxy : 45.43.180.154
start thread_id : 8 avec proxy : 216.19.205.243
start thread_id : 9 avec proxy : 45.43.180.164
start thread_id : 10 avec proxy : 64.137.89.223
start thread_id : 11 avec proxy : 109.196.160.69
start thread_id : 12 avec proxy : 45.43.178.166
start thread_id : 13 avec proxy : 109.196.163.167
start thread_id : 14 avec proxy : 45.43.191.116
start thread_id : 15 avec proxy : 45.43.190.59
start thread_id : 16 avec proxy : 45.43.177.47
start thread_id : 17 avec proxy : 45.43.178.128
start thread_id : 18 avec proxy : 45.43.186.72
start thread_id : 19 avec proxy : 64.137.77.14
start thread_id : 20 avec proxy : 64.137.8.50


In [3]:
list_siret = ['44063603300027',
              '34823356000021',
              '34520018200029',
              '34831006100026',
              '35387462100033',
              '38426795100026',
              '40166810800020',
              '35132899200018',
              '48163234700037',
              '35011228000036']
proxy = {"PROXY_HOST": "161.123.152.67", "PROXY_PORT": "6312", "PROXY_USER": "yedzvwaz", "PROXY_PASS": "5es5islnfwxo"}

len(list_siret)

10

In [5]:
sociteScraping = SociteScraping(siret=list_siret[0], proxy=proxy)

      6 à 9 salariés
      19 995,00 €
      M Bruno CASELLA


In [7]:
from ProxyLoader import ProxyLoader
file_path = r"./proxies/Webshare 100 proxies.txt"
proxy_loader = ProxyLoader(file_path)

for i, siret in enumerate(list_siret):
    sociteScraping = SociteScraping(siret=siret, proxy=proxy_loader.get_proxy(i))
    print(sociteScraping.start_scraping())

44063603300027
      6 à 9 salariés
      19 995,00 €
      M Bruno CASELLA
{'status': True, 'data': {'effectif': '6 à 9 salariés', 'capital': '19 995,00 €', 'gerant': 'M Bruno CASELLA'}}
34823356000021
      10 à 19 salariés
      100000,00 EURO
      M Gerard BERROIR
{'status': True, 'data': {'effectif': '10 à 19 salariés', 'capital': '100000,00 EURO', 'gerant': 'M Gerard BERROIR'}}
34520018200029
      6 à 9 salariés
      7 622,45 €
      M Thiebaut HAXAIRE
{'status': True, 'data': {'effectif': '6 à 9 salariés', 'capital': '7 622,45 €', 'gerant': 'M Thiebaut HAXAIRE'}}
34831006100026
      50 à 99 salariés
      1 100 000,00 €
      M Pascal DARAGON
{'status': True, 'data': {'effectif': '50 à 99 salariés', 'capital': '1 100 000,00 €', 'gerant': 'M Pascal DARAGON'}}
35387462100033
      20 à 49 salariés
      300 000,00 €
      M Frederic THEBAULT
{'status': True, 'data': {'effectif': '20 à 49 salariés', 'capital': '300 000,00 €', 'gerant': 'M Frederic THEBAULT'}}
38426795100026
   

In [1]:
import pandas as pd
df = pd.read_excel("./results_data/grimas_new_version_1.xlsx")
df

,num_page,Titre,SIREN,SIRET,Téléphone,Email,Qualifications,VEN,ANT,Maîtrise Qualité,...,MGTI,CHE,CFLCPT,PAC,MIE,BR,LCPT,address,code_postal,ville
0,1,CASELLA ELECTRICITE,440636033,44063603300027,04 74 61 55 79,andre.casella@wanadoo.fr,MGTI && CHE && LCPT && IRVE,0,0,0,...,1,1,0,0,0,0,1,5 HOTEL D'ENTREPRISE PIERRE BLANCH 330 ALLEE D...,01150,SAINT VULBAS CEDEX
1,1,SERELEC 37,348233560,34823356000021,02 47 59 36 59,contact@serelec37.com,MGTI && IRVE,0,0,0,...,1,0,0,0,0,0,0,45 ROUTE DE VAUZELLES ZI - BP 268,37602,LOCHES CEDEX
2,1,H3E,345200182,34520018200029,03 89 71 90 31,h.3.e@orange.fr,LCPT && IRVE,0,0,0,...,0,0,0,0,0,0,1,ZA 1 ZONE ARTISANALE,68240,FRELAND
3,1,EEP,348310061,34831006100026,03 22 20 14 00,secretariat@eep.fr,MGTI && CFMGTI && SPV && IRVE,0,0,0,...,1,0,0,0,0,0,0,7 B AVENUE ROBERT SCHUMANN BP 70417,80104,ABBEVILLE CEDEX
4,1,SAS THEBAULT,353874621,35387462100033,03 44 81 02 29,contact@thebault-electricite.com,LCPT && MGTI && CHE && CFLCPT && CFMGTI &...,0,0,0,...,1,1,1,0,0,0,1,40 CHEMIN DES TISSERANDS,60112,HERCHIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4665,467,EBE,384267951,38426795100026,06 82 55 06 41,ebe.nathalie@orange.fr,MGTI && LCPT && PAC && IRVE,0,0,0,...,1,0,0,1,0,0,1,401 CHEMIN DES AGRIES LOT NE6,31860,LABARTHE SUR LEZE
4666,467,LIBERSA ELECTRICITE,401668108,40166810800020,03 21 86 24 64,contact@libersaelectricite.fr,LCPT && CHE && IRVE,0,0,0,...,0,1,0,0,0,0,1,29 TER ROUTE DE BOULOGNE,62170,NEUVILLE-SOUS-MONTREUIL
4667,467,GUY CANNARD SA,351328992,35132899200018,0385750613,pm.humbert@guy-cannard.com,IRVE,0,0,0,...,0,0,0,0,0,0,0,12 AV DES TILLEULS,71500,LOUHANS
4668,468,REVERCHON ELECTRICITE,481632347,48163234700037,04 78 69 17 85,sarl.reverchon@wanadoo.fr,LCPT && MGTI && IRVE,0,0,0,...,1,0,0,0,0,0,1,73 RUE MATHIEU DUSSURGEY PARC LYON SUD ILOT B,69190,SAINT-FONS
